In [1]:
import requests
import json
from datetime import datetime
from typing import Dict, List, Union


def get_weather_data(location: str, forecast_days: int = 5) -> Dict:
    """
    Retrieve weather data for a specified location.

    Args:
        location (str): City or location name
        forecast_days (int): Number of days to forecast (1-5)

    Returns:
        dict: Weather data including current conditions and forecast

    Raises:
        ValueError: If input parameters are invalid
        requests.RequestException: If API request fails
        KeyError: If API response structure is unexpected
    """

    # Input validation
    if not location or not isinstance(location, str):
        raise ValueError("Location must be a non-empty string")

    if not isinstance(forecast_days, int) or forecast_days < 1 or forecast_days > 5:
        raise ValueError("Forecast days must be an integer between 1 and 5")

    try:
        # Clean and encode location
        location = location.strip()

        # Make API request
        url = f"https://wttr.in/{location}?format=j1"
        response = requests.get(url, timeout=10)
        response.raise_for_status()

        # Parse JSON response
        data = response.json()

        # Validate API response structure
        if 'current_condition' not in data or 'weather' not in data:
            raise KeyError("Invalid weather data received from API")

        if not data['current_condition'] or not data['weather']:
            raise KeyError("Empty weather data received from API")

        # Process current conditions
        current = data['current_condition'][0]

        # Process forecast data (limit to requested days)
        forecast = data['weather'][:forecast_days]

        # Structure the return data
        weather_data = {
            'location': location,
            'current': {
                'temperature': int(current['temp_C']),
                'temperature_f': int(current['temp_F']),
                'condition': current['weatherDesc'][0]['value'],
                'humidity': int(current['humidity']),
                'wind_speed': int(current['windspeedKmph']),
                'wind_direction': current['winddir16Point'],
                'visibility': int(current['visibility']),
                'pressure': int(current['pressure']),
                'feels_like': int(current['FeelsLikeC']),
                'feels_like_f': int(current['FeelsLikeF']),
                'uv_index': int(current['uvIndex'])
            },
            'forecast': []
        }

        # Process forecast data
        for day in forecast:
            day_data = {
                'date': day['date'],
                'max_temp': int(day['maxtempC']),
                'min_temp': int(day['mintempC']),
                'max_temp_f': int(day['maxtempF']),
                'min_temp_f': int(day['mintempF']),
                'condition': day['hourly'][0]['weatherDesc'][0]['value'],
                'humidity': int(day['hourly'][0]['humidity']),
                'wind_speed': int(day['hourly'][0]['windspeedKmph']),
                'wind_direction': day['hourly'][0]['winddir16Point']
            }
            weather_data['forecast'].append(day_data)

        # Add metadata
        weather_data['timestamp'] = datetime.now().isoformat()
        weather_data['forecast_days'] = len(weather_data['forecast'])

        return weather_data

    except requests.exceptions.RequestException as e:
        raise requests.RequestException(f"Failed to fetch weather data: {str(e)}")
    except (KeyError, IndexError, ValueError) as e:
        raise KeyError(f"Error parsing weather data: {str(e)}")
    except Exception as e:
        raise Exception(f"Unexpected error: {str(e)}")


def display_weather(weather_data: Dict) -> None:
    """
    Display weather data in a formatted way.

    Args:
        weather_data (dict): Weather data from get_weather_data function
    """
    print("=" * 60)
    print(f"WEATHER DATA FOR {weather_data['location'].upper()}")
    print("=" * 60)

    # Current conditions
    current = weather_data['current']
    print(f"\n🌡️  CURRENT CONDITIONS")
    print(f"   Temperature: {current['temperature']}°C ({current['temperature_f']}°F)")
    print(f"   Feels like:  {current['feels_like']}°C ({current['feels_like_f']}°F)")
    print(f"   Condition:   {current['condition']}")
    print(f"   Humidity:    {current['humidity']}%")
    print(f"   Wind:        {current['wind_speed']} km/h {current['wind_direction']}")
    print(f"   Visibility:  {current['visibility']} km")
    print(f"   Pressure:    {current['pressure']} mb")
    print(f"   UV Index:    {current['uv_index']}")

    # Forecast
    print(f"\n📅 {weather_data['forecast_days']}-DAY FORECAST")
    print("-" * 60)

    for i, day in enumerate(weather_data['forecast']):
        day_name = "Today" if i == 0 else day['date']
        print(f"{day_name:12} | {day['condition']:20} | "
              f"High: {day['max_temp']:2}°C | Low: {day['min_temp']:2}°C | "
              f"Humidity: {day['humidity']:2}%")

    print(f"\n📊 Data retrieved: {weather_data['timestamp']}")
    print("=" * 60)


# Example usage and function calls
if __name__ == "__main__":
    # Example 1: Basic usage
    print("Example 1: Getting weather for London")
    try:
        london_weather = get_weather_data("London")
        display_weather(london_weather)
    except Exception as e:
        print(f"Error: {e}")

    print("\n" + "="*80 + "\n")

    # Example 2: Custom forecast days
    print("Example 2: Getting 3-day forecast for New York")
    try:
        nyc_weather = get_weather_data("New York", forecast_days=3)
        display_weather(nyc_weather)
    except Exception as e:
        print(f"Error: {e}")

    print("\n" + "="*80 + "\n")

    # Example 3: Error handling demonstration
    print("Example 3: Error handling")

    # Invalid location
    try:
        invalid_weather = get_weather_data("")
    except ValueError as e:
        print(f"Validation Error: {e}")

    # Invalid forecast days
    try:
        invalid_forecast = get_weather_data("Paris", forecast_days=10)
    except ValueError as e:
        print(f"Validation Error: {e}")

    print("\n" + "="*80 + "\n")

    # Example 4: Multiple locations
    print("Example 4: Multiple locations comparison")
    cities = ["Tokyo", "Sydney", "Mumbai"]

    for city in cities:
        try:
            weather = get_weather_data(city, forecast_days=2)
            print(f"\n{city}: {weather['current']['temperature']}°C, "
                  f"{weather['current']['condition']}")
        except Exception as e:
            print(f"Error getting weather for {city}: {e}")

    print("\n" + "="*80 + "\n")

    # Example 5: Accessing specific data
    print("Example 5: Accessing specific weather data")
    try:
        weather = get_weather_data("Perth")

        # Access current temperature
        current_temp = weather['current']['temperature']
        print(f"Current temperature in Perth: {current_temp}°C")

        # Access tomorrow's forecast
        tomorrow = weather['forecast'][1] if len(weather['forecast']) > 1 else None
        if tomorrow:
            print(f"Tomorrow's high/low: {tomorrow['max_temp']}°C / {tomorrow['min_temp']}°C")

        # Check if it's windy
        wind_speed = weather['current']['wind_speed']
        if wind_speed > 20:
            print(f"It's windy! Wind speed: {wind_speed} km/h")
        else:
            print(f"Light wind: {wind_speed} km/h")

    except Exception as e:
        print(f"Error: {e}")

Example 1: Getting weather for London
WEATHER DATA FOR LONDON

🌡️  CURRENT CONDITIONS
   Temperature: 21°C (70°F)
   Feels like:  21°C (70°F)
   Condition:   Sunny
   Humidity:    35%
   Wind:        11 km/h W
   Visibility:  10 km
   Pressure:    1017 mb
   UV Index:    6

📅 3-DAY FORECAST
------------------------------------------------------------
Today        | Patchy rain nearby   | High: 21°C | Low: 11°C | Humidity: 86%
2025-06-03   | Partly Cloudy        | High: 20°C | Low: 11°C | Humidity: 68%
2025-06-04   | Clear                | High: 18°C | Low:  9°C | Humidity: 74%

📊 Data retrieved: 2025-06-03T01:31:17.618796


Example 2: Getting 3-day forecast for New York
WEATHER DATA FOR NEW YORK

🌡️  CURRENT CONDITIONS
   Temperature: 17°C (63°F)
   Feels like:  17°C (63°F)
   Condition:   Sunny
   Humidity:    45%
   Wind:        12 km/h NW
   Visibility:  16 km
   Pressure:    1017 mb
   UV Index:    6

📅 3-DAY FORECAST
------------------------------------------------------------
Tod

In [2]:
pip install hands-on-ai

   ---------------------------------------- 0.0/18.5 MB ? eta -:--:--
   ----- ---------------------------------- 2.6/18.5 MB 12.5 MB/s eta 0:00:02
   ----------- ---------------------------- 5.2/18.5 MB 15.9 MB/s eta 0:00:01
   ------------- -------------------------- 6.3/18.5 MB 10.4 MB/s eta 0:00:02
   ------------- -------------------------- 6.3/18.5 MB 10.4 MB/s eta 0:00:02
   --------------- ------------------------ 7.3/18.5 MB 8.5 MB/s eta 0:00:02
   ------------------ --------------------- 8.4/18.5 MB 7.5 MB/s eta 0:00:02
   -------------------- ------------------- 9.4/18.5 MB 6.8 MB/s eta 0:00:02
   ----------------------- ---------------- 10.7/18.5 MB 6.5 MB/s eta 0:00:02
   ------------------------------- -------- 14.7/18.5 MB 7.8 MB/s eta 0:00:01
   -------------------------------- ------- 14.9/18.5 MB 7.2 MB/s eta 0:00:01
   --------------------------------- ------ 15.7/18.5 MB 7.3 MB/s eta 0:00:01
   ------------------------------------ --- 16.8/18.5 MB 7.0 MB/s eta 0:00:

In [ ]:
from hands_on_ai.chat import get_response, pirate_bot, coder_bot

# Basic chat usage
response = get_response("Tell me about planets")

# Use a personality bot
pirate_response = pirate_bot("Tell me about sailing ships")

In [3]:
pip install hands-on-ai[dev]

   ---------------------------------------- 0.0/2.7 MB ? eta -:--:--
   ----------------------- ---------------- 1.6/2.7 MB 10.5 MB/s eta 0:00:01
   ---------------------------------------- 2.7/2.7 MB 9.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/3.9 MB ? eta -:--:--
   ---------- ----------------------------- 1.0/3.9 MB 10.1 MB/s eta 0:00:01
   ------------------------------------- -- 3.7/3.9 MB 8.7 MB/s eta 0:00:01
   ---------------------------------------- 3.9/3.9 MB 8.2 MB/s eta 0:00:00
   ---------------------------------------- 0.0/8.7 MB ? eta -:--:--
   ---- ----------------------------------- 1.0/8.7 MB 12.5 MB/s eta 0:00:01
   --------- ------------------------------ 2.1/8.7 MB 6.5 MB/s eta 0:00:02
   -------------- ------------------------- 3.1/8.7 MB 4.9 MB/s eta 0:00:02
   ------------------ --------------------- 3.9/8.7 MB 4.4 MB/s eta 0:00:02
   ------------------- -------------------- 4.2/8.7 MB 4.5 MB/s eta 0:00:02
   ------------------------ --

In [4]:
from hands_on_ai.config import load_config, save_config, get_server_url, get_model

# Get configuration values
config = load_config()  # Returns dict with all config
model = get_model()     # Get default LLM model
server = get_server_url()  # Get Ollama server URL

# Save configuration
config["model"] = "llama3"
save_config(config)

In [5]:
from hands_on_ai.chat import get_response, pirate_bot, coder_bot

# Basic chat usage
response = get_response("Tell me about planets")

# Use a personality bot
pirate_response = pirate_bot("Tell me about sailing ships")

💾 Spinning up the AI core for 'llama3'...


[WARNING] Error during request (attempt 1): HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/generate (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x00000200277F67E0>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))


Oops! I got a little tangled up... Let's try that again 😊


[WARNING] Error during request (attempt 2): HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/generate (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000020027CA2000>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))
[WARNING] Error during request (attempt 1): HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/generate (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000020027CA2900>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))


Pieces of eight! Something went wrong, matey. Trying again!


[WARNING] Error during request (attempt 2): HTTPConnectionPool(host='localhost', port=11434): Max retries exceeded with url: /api/generate (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x0000020027C58140>: Failed to establish a new connection: [WinError 10061] No connection could be made because the target machine actively refused it'))
